# Cheat sheet for Django-salesforce
### Devepoment in Jupyter Notebook

In [1]:
# connect to a database
import os, sys
import django
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "testrunner.settings")
django.setup()
from django.db import connections
HOST = connections['salesforce'].sf_session.auth.instance_url
HOST

attempting authentication to https://test.salesforce.com
successfully authenticated myname@example.com.sandbox


'https://cs7.salesforce.com'

In [2]:
# import models
import salesforce
from salesforce.testrunner.example.models import Account, Contact, Opportunity

`DecimalField` uses a full float accuracy on Salesforce. The number of decimal places is used
only for user input and output

In [3]:
opp = Opportunity.objects.filter(name='test op')[0]
opp.amount = 100 / 3.
opp.save()
qs = Opportunity.objects.filter(name='test op')[:1]
[(x.id, x.name, x.amount, x.probability) for x in qs]

[('006M000000DvyPjIAJ', 'test op', Decimal('33.3333333333333'), Decimal('10'))]

In [5]:
# Find Accounts with a Contact with the first name 'Peter' and as well with the first name 'Jane'
qs = (Account.objects
      .filter(id__in=Contact.objects.filter(first_name='Peter').values('account_id'))
      .filter(id__in=Contact.objects.filter(first_name='Jane').values('account_id'))
      .values('Name')
      )
str(qs.query)

"SELECT Account.Name FROM Account WHERE (Account.Id IN (SELECT Contact.AccountId FROM Contact WHERE Contact.FirstName = 'Peter') AND Account.Id IN (SELECT Contact.AccountId FROM Contact WHERE Contact.FirstName = 'Jane'))"

In [6]:
qs = Contact.objects.filter(account__in=Account.objects.filter(Name='abc')).values('name')
str(qs.query)

"SELECT Contact.Name FROM Contact WHERE Contact.AccountId IN (SELECT Account.Id FROM Account WHERE Account.Name = 'abc')"

In [7]:
# # more low level dict cursor
# dcur = connections['salesforce'].connection.cursor(dict)
# dcur.execute("select Id, Name from Opportunity limit 1")
# dcur.fetchall()

In [4]:
# list cursor
cur = connections['salesforce'].cursor()
cur.execute("select Id, Name from Opportunity limit 1")
cur.fetchall()

[['006M000000DvyPjIAJ', 'test op']]

### Introspection
#### a) Introspection to Django Models

In [8]:
# I don't remember field names and field types in models
Opportunity._meta.fields

(<salesforce.fields.SalesforceAutoField: id>,
 <salesforce.fields.CharField: name>,
 <salesforce.fields.DateField: close_date>,
 <salesforce.fields.CharField: stage>,
 <salesforce.fields.DateTimeField: created_date>,
 <salesforce.fields.DecimalField: amount>,
 <salesforce.fields.DecimalField: probability>)

In [9]:
Opportunity._meta.get_field('amount').decimal_places

2